In [1]:
#pip install "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

In [2]:
import numpy as np
import timeit

In [3]:
import scipy.io as sio

In [4]:
mat = sio.loadmat('../data/AFIRO.mat')
#mat = sio.loadmat('../data/80BAU3B.mat')

In [5]:
A=mat['A']
A.shape

(27, 51)

In [6]:
b=mat['b']
len(b)

27

In [7]:
c=mat['c']
len(c)

51

In [8]:
lbounds=mat['lbounds']
ubounds=mat['ubounds']
print(len(lbounds))
print(len(ubounds))

51
51


In [40]:
def generates_matrix(A, b, blow, bupp, c, compr = True):
    """
    Genera la matriz bajo el supuesto de que se tiene ya capturado los datos de entrada. Genera el problema del tipo:
    `
    Al tipo
    Ax <= b
    Concantenando renglones de A y juntando los limites de las restricciones (multiplicando por -1 donde sea necesario)
    """
    
    if compr:
        A = A.todense()
    
    nrow, ncol = A.shape

    # Concatenar A con matrices identidades de variables sinteticas de forma rbind
    identidad_vars_sint = np.eye(ncol, ncol)
    c = c.T
    pos_ini = len(blow)
    identidad_vars_sint[pos_ini:,:] = -identidad_vars_sint[pos_ini:,:]
    A_new = np.concatenate((A, identidad_vars_sint, c), axis=0)

    # Concatenar variables coeficientes sinteticas
    A_sint = np.eye(nrow + len(blow)*2+1, nrow + len(blow)*2+1)
    pos_row = nrow+1
    pos_col = ncol+1
    A_sint[0:pos_row,0:pos_col] = A_sint[0:pos_row,0:pos_col]*0
    
    # Concatenar A_new con A_sint de formca rbind
    print(A_new.shape)
    print(A_sint.shape)
    A_new = np.c_[A_new, A_sint]
    
    # Juntar cotas de restricciones
    b_new = np.concatenate((b, bupp, -blow, np.array([[0]])), axis=0)
    
    # Concatenar A_new, b_new de forma cbind
    A_new = np.c_[A_new, b_new]
    
    return A_new

# Prueba de paquetería `mex`

In [41]:
from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, \
find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot

from mex.simplex.problem_definition import add_cons, \
constrain, add_obj, obj, maxz, minz

In [42]:
matrix_min_approx = create_matrix(2,5)
matrix_min_approx

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [43]:
constrain(matrix_min_approx,'1,1,E,6')
constrain(matrix_min_approx,'-1,2,E,8')
constrain(matrix_min_approx,'4,3,E,8')
constrain(matrix_min_approx,'1,0,L,8')
constrain(matrix_min_approx,'0,1,G,8')
obj(matrix_min_approx,'-1,-3,0')
matrix_min_approx

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  8.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  8.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  8.],
       [-0., -1.,  0.,  0.,  0.,  0.,  1.,  0., -8.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

In [44]:
A_prueba = np.array([[1,1], [-1,2], [4,3]])
b_prueba = np.array([[6,8,8]]).T
c_prueba = np.array([[1,3]]).T
blow_prueba = np.array([[8]]).T
bupp_prueba = np.array([[8]]).T

In [45]:
generates_matrix(A_prueba, b_prueba, blow_prueba, bupp_prueba, c_prueba, False)

(6, 2)
(6, 6)


array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  8.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  8.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  8.],
       [-0., -1.,  0.,  0.,  0.,  0.,  1.,  0., -8.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

### Prueba del paquete `mex`

In [46]:
mat = sio.loadmat('../data/AFIRO.mat')
#mat = sio.loadmat('../data/80BAU3B.mat')

In [47]:
A = mat['A']
b = mat['b']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
c = mat['c']

In [49]:
print(A.shape)

(27, 51)


In [48]:
tableau = generates_matrix(A, b, lbounds, ubounds, c)

(79, 51)
(130, 130)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 79 and the array at index 1 has size 130

In [19]:
min(matrix[:-1, -1])

-160.0

In [20]:
start = timeit.timeit()

problem_approx = maxz(matrix)

end = timeit.timeit()
print(end - start)


KeyboardInterrupt: 

Comenzó a correr a las $3:11:00$

In [ ]:
start = timeit.timeit()

problem_approx = minz(matrix)

end = timeit.timeit()
print(end - start)
